In [1]:
import numpy as np
import pandas as pd
!pip install --upgrade pip fuzzywuzzy
!pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy.fuzz import partial_ratio
import itertools
!pip install vaex
import vaex

Requirement already up-to-date: pip in /Users/rithviksunku/anaconda3/lib/python3.7/site-packages (20.1.1)
Requirement already up-to-date: fuzzywuzzy in /Users/rithviksunku/anaconda3/lib/python3.7/site-packages (0.18.0)


In [2]:
df = vaex.from_csv('correspondence.csv', usecols=['application_number','correspondence_name_line_1'],low_memory=False)
df

#,application_number,correspondence_name_line_1
0,02107244,JOHN R. UTERMOHLE
1,02565523,MR. EDWARD A. STEEN
2,02568368,JOHN R. UTERMOHLE
3,02602618,PATENT COUNSEL
4,02618355,UNITED STATES DEPARTMENT OF ENERGY
...,...,...
"8,719,693",PCT/US98/26528,"HUFFMAN LAW GROUP, P.C."
"8,719,694",PCT/US99/21785,DICKINSON WRIGHT PLLC
"8,719,695",PCT/US99/29296,"Alchemy-Partners, PC"
"8,719,696",PCT/US99/29645,"HUFFMAN LAW GROUP, P.C."


In [3]:
df1 = vaex.from_csv('firm_modified.csv')
df1

#,Unnamed: 0,rf_id,ee_name
0,0,12800340,CHILDREN'S MEDICAL CENTER CORPORATION
1,1,36340874,PARADYNE CORPORATION
2,2,36340875,PARADYNE CORPORATION
3,3,37290735,"JOVA ENTERPRISES, INC."
4,4,37370606,"'HIRADA STECHNIKAI IPARI KUTATO INTEZET, A CORP...."
...,...,...,...
"5,274,558",5274558,346040462,RAD LABORATORIES INC.
"5,274,559",5274559,346040469,"NANO-RETINA, INC."
"5,274,560",5274560,346040483,CALIFORNIA INSTITUTE OF TECHNOLOGY
"5,274,561",5274561,346040483,UNIVERSITY OF SOUTHERN CALIFORNIA


In [4]:
df['correspondence_name_line_1'] = df.correspondence_name_line_1.str.lower()
df

#,application_number,correspondence_name_line_1
0,02107244,john r. utermohle
1,02565523,mr. edward a. steen
2,02568368,john r. utermohle
3,02602618,patent counsel
4,02618355,united states department of energy
...,...,...
"8,719,693",PCT/US98/26528,"huffman law group, p.c."
"8,719,694",PCT/US99/21785,dickinson wright pllc
"8,719,695",PCT/US99/29296,"alchemy-partners, pc"
"8,719,696",PCT/US99/29645,"huffman law group, p.c."


In [5]:
df['is_law_firm'] = df.correspondence_name_line_1.str.contains('llc')+ df.correspondence_name_line_1.str.contains('partners')+ df.correspondence_name_line_1.str.contains('p.c') + df.correspondence_name_line_1.str.contains('pc') + df.correspondence_name_line_1.str.contains('law') + df.correspondence_name_line_1.str.contains('llp') + df.correspondence_name_line_1.str.contains('lp') + df.correspondence_name_line_1.str.contains('ltd') + df.correspondence_name_line_1.str.contains('patent')+df.correspondence_name_line_1.str.contains('legal')+df.correspondence_name_line_1.str.contains('inc') 
df

#,application_number,correspondence_name_line_1,is_law_firm
0,02107244,john r. utermohle,False
1,02565523,mr. edward a. steen,False
2,02568368,john r. utermohle,False
3,02602618,patent counsel,True
4,02618355,united states department of energy,False
...,...,...,...
"8,719,693",PCT/US98/26528,"huffman law group, p.c.",True
"8,719,694",PCT/US99/21785,dickinson wright pllc,True
"8,719,695",PCT/US99/29296,"alchemy-partners, pc",True
"8,719,696",PCT/US99/29645,"huffman law group, p.c.",True


In [6]:
df.export_csv('correspondence_name_line_1.csv')

In [7]:
df1.export_csv('ee_names.csv')

In [8]:
df = pd.read_csv('correspondence_name_line_1.csv', low_memory=False)
df1 = pd.read_csv('ee_names.csv',low_memory=False)

In [9]:
df1 = df1.dropna()
df = df.dropna()

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct

In [11]:
import string
import re
!pip install ftfy 
from ftfy import fix_text

def ngrams(string, n=3):
    string = fix_text(string)
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower() #make lower case
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) #remove the list of chars defined above
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single space
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [12]:
correspondence = df['correspondence_name_line_1'].unique()
ee_name = df1['ee_name'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix1 = vectorizer.fit_transform(correspondence)
tf_idf_matrix2 =vectorizer.fit_transform(ee_name)

In [13]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [14]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [15]:
matches1 = awesome_cossim_top(tf_idf_matrix1, tf_idf_matrix1.transpose(), 10, 0.80)
matches_df1 = get_matches_df(matches1, correspondence, top=10000)
matches_df1 = matches_df1[matches_df1['similairity'] < 0.99999] # Remove all exact matches
correspondence_dict = pd.Series(matches_df1.right_side.values,index=matches_df1.left_side).to_dict()
df['disam_name'] = df['correspondence_name_line_1'].map(correspondence_dict).fillna(df['correspondence_name_line_1'])
df

,application_number,correspondence_name_line_1,is_law_firm,disam_name
0,02107244,john r. utermohle,False,attn: john r. utermohle
1,02565523,mr. edward a. steen,False,"edward a. steen, esq."
2,02568368,john r. utermohle,False,attn: john r. utermohle
3,02602618,patent counsel,True,patent counsel - patent dept.
4,02618355,united states department of energy,False,united states dept. of energy
5,02645467,paul a. gottlieb,False,paul a. gottlieb a. g. c.
6,02675901,william l. keefauver,False,w. l. keefauver
7,02675903,w. l. keefauver,False,william l. keefauver
8,02682096,office of the associate general counsel (art),False,office of the associate counsel
9,02737085,"raymond j. kenney,esq.",False,"raymond j. kenn,esq."


In [19]:
matches2 = awesome_cossim_top(tf_idf_matrix2, tf_idf_matrix2.transpose(), 10, 0.80)
matches_df2 = get_matches_df(matches2, ee_name, top=10000)
matches_df2 = matches_df2[matches_df2['similairity'] < 0.99999] # Remove all exact matches
ee_dict = pd.Series(matches_df2.right_side.values,index=matches_df2.left_side).to_dict()
df1['disam_name'] = df1['ee_name'].map(ee_dict).fillna(df1['ee_name'])
df1

,Unnamed: 0,rf_id,ee_name,disam_name
0,0,12800340,CHILDREN'S MEDICAL CENTER CORPORATION,CHILDREN'S MEDICAL CENTER
1,1,36340874,PARADYNE CORPORATION,PARADYNE
2,2,36340875,PARADYNE CORPORATION,PARADYNE
3,3,37290735,"JOVA ENTERPRISES, INC.","JOVA ENTERPRISES, INC., A CORP. OF CA"
4,4,37370606,"HIRADA STECHNIKAI IPARI KUTATO INTEZET, A CORP...","HIRADA STECHNIKAI IPARI KUTATO INTEZET, A CORP..."
5,5,37410900,KERLEY CHEMICAL CORPORATION,KERLEY CHEMICAL CORPORATION
6,6,37650757,INDUSTRIE PIRELLI S.P.A.,PIRELLI S.P.A.
7,7,37880668,"LOCKWOOD, RAYMOND M.","LOCKWOOD, RAYMOND M."
8,8,37880670,ROCKWELL INTERNATIONAL CORPORATION,ROCKWELL INTERNATIONAL CORP.
9,9,37880673,TELDEC-TELEFUNKEN-DECCA SCHALLPLATTEN GMBH,TELDEC-TELEFUNKEN-DECCA SCHALLPLATTEN GMBH


In [20]:
df['disam_id'] = df.disam_name.map(hash)
df1['disam_id'] = df1.disam_name.map(hash)

In [17]:
df.head(50)

,application_number,correspondence_name_line_1,is_law_firm,disam_name,disam_id
0,02107244,john r. utermohle,False,attn: john r. utermohle,-1647916547179011215
1,02565523,mr. edward a. steen,False,"edward a. steen, esq.",-802377925817009679
2,02568368,john r. utermohle,False,attn: john r. utermohle,-1647916547179011215
3,02602618,patent counsel,True,patent counsel - patent dept.,2318155992507833048
4,02618355,united states department of energy,False,united states dept. of energy,5988365442746328720
5,02645467,paul a. gottlieb,False,paul a. gottlieb a. g. c.,2875278055825636018
6,02675901,william l. keefauver,False,w. l. keefauver,-7511442581082519454
7,02675903,w. l. keefauver,False,william l. keefauver,3689766024981991878
8,02682096,office of the associate general counsel (art),False,office of the associate counsel,-425424279860912879
9,02737085,"raymond j. kenney,esq.",False,"raymond j. kenn,esq.",803235408900661078


In [21]:
df1.head(50)

,Unnamed: 0,rf_id,ee_name,disam_name,disam_id
0,0,12800340,CHILDREN'S MEDICAL CENTER CORPORATION,CHILDREN'S MEDICAL CENTER,-8061836815952169505
1,1,36340874,PARADYNE CORPORATION,PARADYNE,2676085245486288441
2,2,36340875,PARADYNE CORPORATION,PARADYNE,2676085245486288441
3,3,37290735,"JOVA ENTERPRISES, INC.","JOVA ENTERPRISES, INC., A CORP. OF CA",-8411236392303285112
4,4,37370606,"HIRADA STECHNIKAI IPARI KUTATO INTEZET, A CORP...","HIRADA STECHNIKAI IPARI KUTATO INTEZET, A CORP...",-5204464025172734193
5,5,37410900,KERLEY CHEMICAL CORPORATION,KERLEY CHEMICAL CORPORATION,3611821189359767227
6,6,37650757,INDUSTRIE PIRELLI S.P.A.,PIRELLI S.P.A.,7332056629914962109
7,7,37880668,"LOCKWOOD, RAYMOND M.","LOCKWOOD, RAYMOND M.",-2695776874084097874
8,8,37880670,ROCKWELL INTERNATIONAL CORPORATION,ROCKWELL INTERNATIONAL CORP.,-2995935799954998525
9,9,37880673,TELDEC-TELEFUNKEN-DECCA SCHALLPLATTEN GMBH,TELDEC-TELEFUNKEN-DECCA SCHALLPLATTEN GMBH,-8953842303825474738


In [22]:
df.to_csv('correspondence_final.csv')
df1.to_csv('ee_name_final.csv')